In [1]:
!pip install datasets
from datasets import load_dataset
raw_datasets=load_dataset("kde4",lang1="en",lang2="fr")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/210173 [00:00<?, ? examples/s]

In [2]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

In [3]:
split_datasets=raw_datasets["train"].train_test_split(train_size=0.9,seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [4]:
#renaming the test key to validation
split_datasets["validation"]=split_datasets.pop("test")

In [5]:
split_datasets["train"][1]["translation"]

{'en': 'Default to expanded threads',
 'fr': 'Par défaut, développer les fils de discussion'}

In [6]:
from transformers import pipeline
pre_model="Helsinki-NLP/opus-mt-en-fr"
translator=pipeline("translation",model=pre_model)
translator("Default to expanded threads.")

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'translation_text': 'Par défaut pour les threads élargis.'}]

In [7]:
#fine tuning the model

In [8]:
#Data Processing for Translation
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(pre_model,return_tensors="tf")

In [9]:
en_sentence=split_datasets["train"][1]["translation"]["en"]
fr_sentence=split_datasets["train"][1]["translation"]["fr"]
inputs=tokenizer(en_sentence,text_target=fr_sentence)
inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]}

In [10]:
wrong_targets=tokenizer(fr_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁Par', '▁dé', 'f', 'aut', ',', '▁dé', 've', 'lop', 'per', '▁les', '▁fil', 's', '▁de', '▁discussion', '</s>']
['▁Par', '▁défaut', ',', '▁développer', '▁les', '▁fils', '▁de', '▁discussion', '</s>']


In [11]:
#defining the preprocessing function
max_lenght=128
def preprocess_function(examples):
  inputs=[ex["en"] for ex in examples["translation"]]
  targets=[ex["fr"] for ex in examples["translation"]]
  return tokenizer(inputs,text_target=targets,max_length=128,truncation=True)

In [12]:
tokenized_datasets=split_datasets.map(preprocess_function,batched=True,remove_columns=split_datasets["train"].column_names,)

Map:   0%|          | 0/189155 [00:00<?, ? examples/s]

Map:   0%|          | 0/21018 [00:00<?, ? examples/s]

In [13]:
#fine tuning
from transformers import TFAutoModelForSeq2SeqLM
model=TFAutoModelForSeq2SeqLM.from_pretrained(pre_model,from_pt=True)

All PyTorch model weights were used when initializing TFMarianMTModel.

All the weights of TFMarianMTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [14]:
#Data Collation
#here we will need a data collator for dynamic batching
from transformers import DataCollatorForSeq2Seq
data_collator=DataCollatorForSeq2Seq(tokenizer,model=model,return_tensors="tf")

In [15]:
#example of the above is
batch=data_collator([tokenized_datasets["train"][i] for i in range(1,3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [16]:
batch["labels"]

<tf.Tensor: shape=(2, 16), dtype=int64, numpy=
array([[  577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,
         -100,  -100,  -100,  -100,  -100,  -100,  -100],
       [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,
          817,   550,  7032,  5821,  7907, 12649,     0]])>

In [17]:
batch["decoder_input_ids"]

<tf.Tensor: shape=(2, 16), dtype=int64, numpy=
array([[59513,   577,  5891,     2,  3184,    16,  2542,     5,  1710,
            0, 59513, 59513, 59513, 59513, 59513, 59513],
       [59513,  1211,     3,    49,  9409,  1211,     3, 29140,   817,
         3124,   817,   550,  7032,  5821,  7907, 12649]])>

In [18]:
for i in range(1,3):
  print(tokenized_datasets["train"][i]["labels"])

[577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]
[1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 550, 7032, 5821, 7907, 12649, 0]


In [19]:
tf_train_dataset=model.prepare_tf_dataset(
    tokenized_datasets["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)
tf_eval_dataset=model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

Bleu Metric:-
The BLEU score evaluates how close the translations are to their labels. It does not measure the intelligibility or grammatical correctness of the model’s generated outputs, but uses statistical rules to ensure that all the words in the generated outputs also appear in the targets.

In [20]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 9.1 MB/s eta 0:00:00


In [21]:
!pip install evaluate
import evaluate
metric=evaluate.load("sacrebleu")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.4 MB/s eta 0:00:00


In [22]:
#example for using this bleu score

#example of compatible sentences
predictions = ["This plugin lets you translate web pages between several languages automatically."]
references = [["This plugin allows you to automatically translate web pages between several languages."]]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [23]:
#example of incompatile sentences
predictions = ["This This This This"]
references = [["This plugin allows you to automatically translate web pages between several languages."]]
metric.compute(predictions=predictions, references=references)

{'score': 1.683602693167689,
 'counts': [1, 0, 0, 0],
 'totals': [4, 3, 2, 1],
 'precisions': [25.0, 16.666666666666668, 12.5, 12.5],
 'bp': 0.10539922456186433,
 'sys_len': 4,
 'ref_len': 13}

In [24]:
#as we can see above higher the score better is the comptibility between the sentences

In [32]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

generation_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model,return_tensors="tf",pad_to_multiple_of=128)
tf_generate_dataset=model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    collate_fn=generation_data_collator,
    shuffle=False,
    batch_size=8
)



def generate_with_xla(batch):
    return model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=128,
    )

@tf.function(jit_compile=True)
def compute_metrics():
    all_preds = []
    all_labels = []

    for batch, labels in tqdm(tf_generate_dataset):
        predictions = generate_with_xla(batch)
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = labels.numpy()
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = [pred.strip() for pred in decoded_preds]
        decoded_labels = [[label.strip()] for label in decoded_labels]
        all_preds.extend(decoded_preds)
        all_labels.extend(decoded_labels)

    result = metric.compute(predictions=all_preds, references=all_labels)
    return {"bleu": result["score"]}

In [26]:
#creating the model
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [27]:

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    epochs=num_epochs,
)

Epoch 1/3
5911/5911 [==============================] - 3567s 596ms/step - loss: 1.0620 - val_loss: 0.8781
Epoch 2/3
5911/5911 [==============================] - 3539s 599ms/step - loss: 0.7982 - val_loss: 0.8188
Epoch 3/3
5911/5911 [==============================] - 3486s 590ms/step - loss: 0.6861 - val_loss: 0.8043


In [31]:
from transformers import pipeline

# Replace this with your own checkpoint
translator = pipeline("translation", model=model,tokenizer=tokenizer)
translator("Default to expanded threads")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'translation_text': 'Par défaut, développer les fils de discussion'}]

In [34]:
translator("Hello, I am Rahul")

[{'translation_text': 'Bonjour, je suis Rahul'}]

In [35]:
translator("I am a good person.")

[{'translation_text': 'Je suis une bonne personne.'}]